# How to use caching and buffering

Caching and buffering are two techniques that can improve the performance and efficiency of your database queries. Caching means storing the results of a query in memory for faster access later, while buffering means reading the data from the database in chunks rather than all at once.

Dapper does not include any built-in data caching features, but you can use any external caching component, such as the MemoryCache class in the <code>Microsoft.Extensions.Caching.Memory</code> namespace. You can also configure the buffer parameter in the <code>Query</code> method to control how Dapper reads the data from the database. By default, the buffer parameter is set to true, which means Dapper will load the entire result set into a <code>&lt;List&gt;<T></code> before returning it to you. This can be useful if you want to iterate over the data multiple times, or avoid conflicts with subsequent operations on the same connection. However, if you have a large or complex query, you may want to set the buffer parameter to false, which means Dapper will stream the data as you consume it. This can reduce the memory footprint and latency of your query, but it also means you can only iterate over the data once, and you need to be careful not to use the same connection for other commands while iterating.

Here is a complete example of how to use caching and buffering with C# and Dapper, using the AdventureWorks2022 database.

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Microsoft.Extensions.Caching.Memory"
#r "nuget:Dapper"

using Microsoft.Data.SqlClient;
using Microsoft.Extensions.Caching.Memory;
using System;
using Dapper;

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.2 Microsoft.Extensions.Caching.Memory, 8.0.0

In [3]:
public class Product
{
    public int ProductID { get; set; }
    public string Name { get; set; }
    public decimal ListPrice { get; set; }
}

In [4]:
// A memory cache instance
static readonly IMemoryCache Cache = new MemoryCache(new MemoryCacheOptions());

// A query to get all products from the database
const string query = "SELECT ProductID, Name, ListPrice FROM Production.Product WHERE ListPrice >= 3000";

// A method to get the products from the cache or the database
private static async Task<IEnumerable<Product>> GetProductsAsync()
{
    // Try to get the products from the cache
    if (Cache.TryGetValue("Products", out IEnumerable<Product> products))
    {
        Console.WriteLine("Products retrieved from cache.");
        return products;
    }

    // If the cache is empty, get the products from the database
    Console.WriteLine("Products retrieved from database.");
    using (var connection = new SqlConnection(connectionString))
    {
        // Use buffered: false to stream the data from the database
        products = connection.Query<Product>(query, buffered: false);
        // products = await connection.QueryAsync<Product>(query, buffered: false);

        // Store the products in the cache with an expiration time of 10 minutes
        Cache.Set("Products", products, TimeSpan.FromMinutes(10));

        // Return the products as a list
        return products.ToList();
    }
}

// Get the products from the cache or the database
var products = await GetProductsAsync();

// Print the products to the console
foreach (var product in products)
{
    Console.WriteLine($"{product.ProductID}: {product.Name} - {product.ListPrice:C}");
}

Products retrieved from database.
749: Road-150 Red, 62 - ¤3,578.27
750: Road-150 Red, 44 - ¤3,578.27
751: Road-150 Red, 48 - ¤3,578.27
752: Road-150 Red, 52 - ¤3,578.27
753: Road-150 Red, 56 - ¤3,578.27
771: Mountain-100 Silver, 38 - ¤3,399.99
772: Mountain-100 Silver, 42 - ¤3,399.99
773: Mountain-100 Silver, 44 - ¤3,399.99
774: Mountain-100 Silver, 48 - ¤3,399.99
775: Mountain-100 Black, 38 - ¤3,374.99
776: Mountain-100 Black, 42 - ¤3,374.99
777: Mountain-100 Black, 44 - ¤3,374.99
778: Mountain-100 Black, 48 - ¤3,374.99
